In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"

import os

import subprocess

## IMPORTANT: Make sure that "Uniandes_Framework" is in .gitignore
framework_path = "Uniandes_Framework"

if os.path.exists(framework_path):
    # Pull updates if the framework is already cloned
    try:
        subprocess.run(["git", "-C", framework_path, "pull"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while pulling updates from the framework: {e}")
else:
    # Clone the framework if it is not already cloned
    try:
        subprocess.run(["git", "clone", "git@github.com:Phenomenology-group-uniandes/Uniandes_Framework.git"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while cloning the framework: {e}")
        
from Uniandes_Framework.heatmaps_utilities.functions import smooth, plot_heatmap

In [ ]:
def approx_global_sig(sig: np.array, bkg: np.array, N: float = 0.0) -> float:
    """
    Calculates the statistical significance of a signal over background in a given dataset using a modified version of
    the formula (S -N sqrt(B))/sqrt(S+B), where S is the number of signal events, B is the number of background events,
    and N is the expected number of background events in the signal region.

    Parameters:
    sig (np.array): 1D array containing the number of signal events in each bin of the dataset.
    bkg (np.array): 1D array containing the number of background events in each bin of the dataset.
    N (float): Expected number of background events in the signal region. Default value is 0.0.

    Returns:
    float: The statistical significance of the signal over background in the dataset.
    """

    # calculate weight factor w for each bin
    w = np.log(1. + sig/(bkg + 1e-9))

    # calculate intermediate quantities
    s_w = sig * w
    b_w = bkg * w
    s_ww = sig * w ** 2
    b_ww = bkg * w ** 2

    # calculate numerator and denominator of modified formula
    num = np.sum(s_w) - N * np.sqrt(np.sum(b_ww))
    den = np.sqrt(np.sum(s_ww + b_ww))

    # calculate statistical significance and return it
    return num / den

In [ ]:
signal = 'tau_tau'

bkgs =  ['tbart', 'V+jets', 'stop', 'Diboson']
Masses = ["1000", '1250', '1500', '1750', '2000', '2250', '2500']
betards = ['wRHC', 'woRHC']
suffix_by_betard = {'wRHC': '', 'woRHC': '_wo_RHC'}

channels = {'hadronic_dLQ': 'hadronic_Tau_Tau_b_b',
           'hadronic_sLQ': 'hadronic_Tau_Tau_b',
           'hadronic_non-resonant': 'hadronic_Tau_Tau',
           'semileptonic_dLQ': 'semileptonic_Tau_Tau_b_b',
           'semileptonic_sLQ': 'semileptonic_Tau_Tau_b',
           'semileptonic_non-resonant': 'semileptonic_Tau_Tau'}

XS_Dict = {}

for signal in ['ta_ta', 'zp_tau_tau']:
    Path = os.path.join(os.path.dirname(os.getcwd()), "01_signal_production", 'xs_signals_woRHC', signal, 'XS_Matrix.csv')
    
    XS_Dict[signal] = pd.read_csv(Path, index_col= 0)
    XS_Dict[signal].columns = [float(i) for i in XS_Dict[signal].columns]
    XS_Dict[signal].index = [float(i) for i in XS_Dict[signal].index]

In [ ]:
XS_Dict[signal].index

In [ ]:
gU = 1.5
gU_name = str(gU).replace('.','_')

In [ ]:
Significances = {'wRHC': {}, 'woRHC': {}}

#non-res:
for betard in betards:
    Significances[betard]['non-res'] = {}
    
    for Mass in Masses:
        Significances[betard]['non-res'][float(Mass)] = {} 

        for coupling in XS_Dict['zp_tau_tau'].index:
            
            Matrix_Signal = np.zeros([11,6])
            Matrix_BKG = np.zeros([11,6])
            for n, channel in enumerate(channels):
                
                Efficiences = pd.read_csv(os.path.join(f'Data_LQS2023/Efficiences', f'{channel}.csv'), index_col= 0)
                Efficience = Efficiences[f'Tau_Tau{suffix_by_betard[betard]}_{Mass}']['DeltaR > 0.3']
                XS_Total = XS_Dict['ta_ta'][float(Mass)][gU] + XS_Dict['zp_tau_tau'][float(Mass)][coupling]
                Luminosity = 137*1000
                N_events = Efficience*XS_Total*Luminosity
                
                path_to_txt = os.path.join(f'Data_LQS2023/Histograms_{betard}', f'M{Mass}', channels[channel], f"high_per_bin_tau_tau.txt")
                high_per_bin = np.loadtxt(path_to_txt)
                high_per_bin = high_per_bin/sum(high_per_bin)                

                Matrix_Signal[:, n] += high_per_bin*N_events

                for bkg in bkgs:
                    path_to_txt = os.path.join(f'Data_LQS2023/Histograms_{betard}', f'M{Mass}', channels[channel], f"high_per_bin_{bkg}.txt")
                    Matrix_BKG[:, n] += np.loadtxt(path_to_txt)                

            Signal_Data = np.asarray(Matrix_Signal.reshape((1,66)))
            BKG_Data = np.asarray(Matrix_BKG.reshape((1,66)))    
            Significances[betard]['non-res'][float(Mass)][coupling/gU] = approx_global_sig(Signal_Data, BKG_Data)

In [ ]:
for betard in betards:
    file_name = f'Significances/Significance_Table_13TeV_L137_non-res_{betard}_with_gU_{gU_name}.csv'
    pd.DataFrame.from_dict(Significances[betard]['non-res']).to_csv(file_name)

In [ ]:
betard_title = {'wRHC': r'$, \mathbf{\beta_{R} = -1}$', 'woRHC': r'$, \mathbf{\beta_{R} = 0}$'}

# ubicaciones = {'non-res_wRHC': [[1600/1000,1], [1600/1000, 1.2], [1600/1000, 1.5]],
#                'non-res_woRHC': [[1600/1000,0.8], [1600/1000, 1.25], [1600/1000, 1.5]], 
#               }

def Calcular_g_U(c_U, M):
    v = 246
    return 2*np.sqrt(c_U)*M/v

for betard in betards:
    
    Data = pd.DataFrame.from_dict(Significances[betard]['non-res'])
    Data.columns = [float(column)/1000 for column in Data.columns] #TeV

    Data_interpolate = smooth(Data, log = True)
    # Data_interpolate = np.log10(Data)

    fig, ax, curves = plot_heatmap(Data_interpolate,
                           level_curves = {np.log10(1.69) : r'$1.69 \sigma$', np.log10(3): r'$3 \sigma$', np.log10(5): r'$5 \sigma$'},
                           # curves_labels_locations= ubicaciones[f'non-res_{betard}'],
                           title_right = r'$\mathbf{\sqrt{s} = 13 TeV , L = 137 fb^{-1}, g_{U} = ' + f'{gU}' + '}$' + betard_title[betard] ,
                           x_label = r'$M_{U}$ [TeV]', 
                           y_label = r'$g_{Z^{\prime}} / g_U$',
                           cbar_label = r'$\log_{10}$(Significance)',
                          )        

    file_name = f'Significances/Significance_Heatmap_13TeV_L137_non-res_{betard}_with_gU_{gU_name}.png'
    plt.savefig(file_name, bbox_inches='tight')

In [ ]:
%%capture

sigma_curves = {}
for betard in betards:
    sigma_curves[betard] = {}
    
for betard in betards:
    
    Data = pd.DataFrame.from_dict(Significances[betard]['non-res'])
    Data.columns = [float(column)/1000 for column in Data.columns] #TeV

    Data_interpolate = smooth(Data, log = True)
    fig, ax, sigma_curves[betard]['non-res'] = plot_heatmap(Data_interpolate, level_curves = {np.log10(1.69) : '' , np.log10(3): '', np.log10(5): ''})    

In [ ]:
#let's save sigma_curves in order to plot curves in other jupyter notebook:
sigma_curves

In [ ]:
# import pickle

# file = open("Significances/sigma_curves_137.pkl","wb")
# pickle.dump(sigma_curves,file)
# file.close()